In [1]:
import numpy as np

In [2]:
import keras
shakespeare_url = "https://homl.info/shakespeare"
filepath = keras.utils.get_file("shakespeare.txt",shakespeare_url)
with open(filepath) as f:
  shakespeare_text = f.read()

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [3]:
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

In [4]:
tokenizer.texts_to_sequences(["First"])

[[20, 6, 9, 8, 3]]

In [5]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

In [6]:
max_id = len(tokenizer.word_index)
dataset_size = tokenizer.document_count

In [7]:
import numpy as np
[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text]))-1

In [8]:
import tensorflow as tf
train_size = dataset_size*90//100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])

In [9]:
n_steps = 100
window_length = n_steps+1
dataset = dataset.window(window_length,shift=1,drop_remainder=True)

In [10]:
dataset = dataset.flat_map(lambda window: window.batch(window_length))

In [11]:
batch_size=32
dataset = dataset.shuffle(10000).batch(batch_size)
dataset = dataset.map(lambda windows:(windows[:,:-1],windows[:,1:]))

In [12]:
dataset = dataset.map(
    lambda X_batch,Y_batch: (tf.one_hot(X_batch,depth=max_id),Y_batch)
)
dataset = dataset.prefetch(1)

In [13]:
model = keras.models.Sequential([
    keras.layers.GRU(128,return_sequences=True,input_shape=[None,max_id],dropout=0.2,recurrent_dropout=0.2),
    keras.layers.GRU(128,return_sequences=True,dropout=0.2,recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id,activation="softmax"))
])

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
model.compile(loss="sparse_categorical_crossentropy",optimizer="adam")
history = model.fit(dataset,epochs=20)

Epoch 1/20
    260/Unknown 46s 167ms/step - loss: 2.9439

In [ ]:
def preprocess(texts):
  X = np.array(tokenizer.texts_to_sequences(texts))-1
  return tf.one_hot(X,max_id)

In [ ]:
X_new = preprocess(["how are yo"])
Y_pred = model.predict_classes(X_new)
tokenizer.sequences_to_texts(Y_pred+1)[0][-1]

In [ ]:
def next_char(text,temperature=1):
  X_new = preprocess([text])
  y_proba = model.predict(X_new)[0,-1:,:]
  rescaled_logits = tf.math.log(y_proba)/temperature
  char_id = tf.random.categorical(rescaled_logits,num_samples=1)+1
  return tokenizer.sequences_to_texts(char_id.numpy())[0]

In [ ]:
def complete_text(text,nchars=50,temperature=1):
  for _ in range(n_chars):
    text += next_char(text,temperature)
  return text

In [ ]:
print(complete_text("t",temperature=0.2))